# Working examples

## Tools

* [Julia 1.7.2](https://julialang.org/downloads/)
* [ConstraintSolver.jl](https://wikunia.github.io/ConstraintSolver.jl/stable/)
* [MiniZinc](https://www.minizinc.org/)
* [VSCode](https://code.visualstudio.com/)
    * [Jupyter Notebooks in VS Code](https://code.visualstudio.com/docs/datascience/jupyter-notebooks)
    * [Julia in VS Code](https://code.visualstudio.com/docs/languages/julia)

## Propagation mechanism

I will present how constraint programming works for the following feasibility problem:

* We have three variables: $x,y \in\{0,...,9\}$ and $z \in\{0...18\}$
* We will play with the following constraints:
    1. $x + y = z$
    2. $z = 13$
    3. $y < x-3$
* Task is to find all solutions that respect given constraints.

In [2]:
using ConstraintSolver
const CS=ConstraintSolver
using JuMP

### First constraint $x + y = z$

In [3]:
function first_constraint()
    m = Model(optimizer_with_attributes(CS.Optimizer, "backtrack"=>false))

    @variable(m, 0 <= x <= 9, Int)
    @variable(m, 0 <= y <= 9, Int)
    @variable(m, 0 <= z <= 18, Int)

    @constraint(m, z == x+y)

    optimize!(m)

    status = JuMP.termination_status(m)
    println("Solving ended with $(status) status.")
    if status in [OPTIMAL, OTHER_LIMIT]
        for v in [x, y, z]
            println(name(v), "=>", CS.values(m, v))
        end
    else
        println("No solution found.")
    end
end

first_constraint();

# Variables: 3
# Constraints: 1
 - # Equality: 1

Solving ended with OTHER_LIMIT status.
x=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
z=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


┌ Info: Backtracking is turned off.
└ @ ConstraintSolver /root/.julia/packages/ConstraintSolver/PGUiU/src/ConstraintSolver.jl:808


### Constraint $z = 13$

Solver was not able to reduce domains. Let's see what happens if I add next constraint.

In [4]:
function second_constraint()
    m = Model(optimizer_with_attributes(CS.Optimizer, "backtrack"=>false))

    @variable(m, 0 <= x <= 9, Int)
    @variable(m, 0 <= y <= 9, Int)
    @variable(m, 0 <= z <= 18, Int)

    @constraint(m, z == x+y)
    @constraint(m, z == 13)

    optimize!(m)

    status = JuMP.termination_status(m)
    println("Solving ended with $(status) status.")
    if status in [OPTIMAL, OTHER_LIMIT]
        for v in [x, y, z]
            println(name(v), "=>", CS.values(m, v))
        end
    else
        println("No solution found.")
    end
end
second_constraint();

# Variables: 3
# Constraints: 1
 - # Equality: 1

Solving ended with OTHER_LIMIT status.
x=>[9, 8, 7, 6, 4, 5]
y=>[9, 8, 7, 6, 4, 5]
z=>[13]


┌ Info: Backtracking is turned off.
└ @ ConstraintSolver /root/.julia/packages/ConstraintSolver/PGUiU/src/ConstraintSolver.jl:808


### Constraint $x < y - 4$

Now solver was able to deduce that variables $x$ and $y$ must be greater than $4$.
Let's see what happens if we add more constraints.

In [5]:
function third_constraint()
    m = Model(optimizer_with_attributes(CS.Optimizer, "backtrack"=>false))

    @variable(m, 0 <= x <= 9, Int)
    @variable(m, 0 <= y <= 9, Int)
    @variable(m, 0 <= z <= 18, Int)


    @constraint(m, z == x+y)
    @constraint(m, z == 13)
    @constraint(m, y < x - 4)
    optimize!(m)

    status = JuMP.termination_status(m)
    println("Solving ended with $(status) status.")
    if status in [OPTIMAL, OTHER_LIMIT]
        for v in [x, y, z]
            println(name(v), "=>", CS.values(m, v))
        end
    else
        println("No solution found.")
    end
end

third_constraint();

# Variables: 3
# Constraints: 2
 - # Equality: 1
 - # Inequality: 1

Solving ended with OPTIMAL status.
x=>[9]
y=>[4]
z=>[13]


### Summary

As you could see the solver was able to reduce domains. Even to the solution just using propagation mechanizm.

## Labeling variables

Now I will present how we can deal with the situation when propagation mechanism was not
able to find solution.


Suppose we wish to schedule six one-hour lectures, v1, v2, v3, v4, v5, v6.
  Among the the potential audience there are people who **wish to hear both**

   - v1 and v2
   - v1 and v4
   - v3 and v5
   - v2 and v6
   - v4 and v5
   - v5 and v6
   - v1 and v6

  How many hours are necessary in order that the lectures can be given
  without clashes?

*Based on [http://hakank.org/julia/constraints/lectures.jl](http://hakank.org/julia/constraints/lectures.jl)*

### First we will create a model for the lecture scheduling problem.

In [6]:
function labeling_variables_1()
    problem = [
        (1, 2),
        (1, 4),
        (3, 5),
        (2, 6),
        (4, 5),
        (5, 6),
        (1, 6)
    ]
    n = size(problem)[1]

    m = Model(optimizer_with_attributes(CS.Optimizer, "backtrack"=>false))

    @variable(m, 1 <= lecture_time[1:n] <= n, Int)
    @variable(m, 1 <= max_t <= n, Int)

    for (lecture_a, lecture_b) in problem
        @constraint(m, lecture_time[lecture_a] != lecture_time[lecture_b])
    end

    @constraint(m, lecture_time .<= max_t)

    @objective(m, Min, max_t)

    optimize!(m)

    status = JuMP.termination_status(m)
    println("Solving ended with $(status) status.")
    if status in [OPTIMAL, OTHER_LIMIT]
        println(name(max_t), "=>", CS.values(m, max_t))
        for i in 1:n
            println(name(lecture_time[i]), "=>", CS.values(m, lecture_time[i]))
        end
    else
        println("No solution found.")
    end
end

labeling_variables_1 (generic function with 1 method)

Let's see if solver can reduce domain of variables.

In [7]:
labeling_variables_1()

# Variables: 8
# Constraints: 14
 - # Inequality: 7
 - # Notequal: 7

Added 1 new constraints
Solving ended with OTHER_LIMIT status.
max_t=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[1]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[2]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[3]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[4]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[5]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[6]=>[1, 2, 3, 4, 5, 6, 7]
lecture_time[7]=>[1, 2, 3, 4, 5, 6, 7]


┌ Info: Backtracking is turned off.
└ @ ConstraintSolver /root/.julia/packages/ConstraintSolver/PGUiU/src/ConstraintSolver.jl:808


As you can see solver was not able to reduce domains of any vairable

### Now we will add labeling phase and compare two value selection strategies:

* Start with smallest value possible
* Start with larget value possible

We will use default variable selection process that prioritize variables that are in objective formula, how often they lead to infeasibility and size of their domains.
More info you can find [here](https://wikunia.github.io/ConstraintSolver.jl/stable/options/#:~:text=Options%3A-,%3AIMPS,-%3D%3E%20Infeasible%20and%20Minimum)

In [8]:
function labeling_variables_2(branch_split)
    problem = [
        (1, 2),
        (1, 4),
        (3, 5),
        (2, 6),
        (4, 5),
        (5, 6),
        (1, 6)
    ]
    n = size(problem)[1]

    m = Model(optimizer_with_attributes(CS.Optimizer,
                    "traverse_strategy"=>:BFS,
                    "branch_strategy"=>:IMPS,
                    "branch_split"=>branch_split,
                    "backtrack"=>true))

    @variable(m, 1 <= lecture_time[1:n] <= n, Int)
    @variable(m, 1 <= max_t <= n, Int)

    for (lecture_a, lecture_b) in problem
        @constraint(m, lecture_time[lecture_a] != lecture_time[lecture_b])
    end

    @constraint(m, lecture_time .<= max_t)

    @objective(m, Min, max_t)

    optimize!(m)

    status = JuMP.termination_status(m)
    println("Solving ended with $(status) status.")
    if status in [OPTIMAL, OTHER_LIMIT]
        println(name(max_t), "=>", CS.values(m, max_t))
        for i in 1:n
            println(name(lecture_time[i]), "=>", CS.values(m, lecture_time[i]))
        end
    else
        println("No solution found.")
    end
end

labeling_variables_2 (generic function with 1 method)

### Smallest values first

In [9]:
@time labeling_variables_2(:Smallest)

# Variables: 8
# Constraints: 14
 - # Inequality: 7
 - # Notequal: 7

Added 1 new constraints
  #Open    #Closed       Incumbent           Best Bound      Time [s] 
    1         1              -                  2.00           0.14   
    1         5              -                  3.00           0.38   
    7         13            3.00                3.00           0.39   
Solving ended with OPTIMAL status.
max_t=>[3]
lecture_time[1]=>[3]
lecture_time[2]=>[2]
lecture_time[3]=>[1]
lecture_time[4]=>[1]
lecture_time[5]=>[2]
lecture_time[6]=>[1]
lecture_time[7]=>[1]
  0.450356 seconds (1.25 M allocations: 66.559 MiB, 3.32% gc time, 99.06% compilation time)


### Biggest values first

In [10]:
@time labeling_variables_2(:Biggest)

# Variables: 8
# Constraints: 14
 - # Inequality: 7
 - # Notequal: 7

Added 1 new constraints
  #Open    #Closed       Incumbent           Best Bound      Time [s] 
    2         0              -                  1.00           0.04   
    6         6              -                  2.00           0.04   
    5         9              -                  3.00           0.04   
    12        18            3.00                3.00           0.04   
Solving ended with OPTIMAL status.
max_t=>[3]
lecture_time[1]=>[3]
lecture_time[2]=>[2]
lecture_time[3]=>[1]
lecture_time[4]=>[1]
lecture_time[5]=>[2]
lecture_time[6]=>[1]
lecture_time[7]=>[1]
  0.039031 seconds (60.51 k allocations: 3.227 MiB, 94.27% compilation time)


### Mean value to split domain in half

In [11]:
@time labeling_variables_2(:InHalf)

# Variables: 8
# Constraints: 14
 - # Inequality: 7
 - # Notequal: 7

Added 1 new constraints
  #Open    #Closed       Incumbent           Best Bound      Time [s] 
    2         0              -                  1.00           0.04   
    3         3              -                  2.00           0.04   
    2         6              -                  3.00           0.04   
    10        16            3.00                3.00           0.04   
Solving ended with OPTIMAL status.
max_t=>[3]
lecture_time[1]=>[3]
lecture_time[2]=>[2]
lecture_time[3]=>[1]
lecture_time[4]=>[1]
lecture_time[5]=>[2]
lecture_time[6]=>[1]
lecture_time[7]=>[1]
  0.040740 seconds (118.16 k allocations: 6.160 MiB, 94.41% compilation time)


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*